In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seml
import pandas as pd
import json
from collections import defaultdict
from functools import reduce
import seaborn as sns
from itertools import product

In [2]:
collection_name = 'week13_densities_cora_ml'
collection = seml.database.get_collection(collection_name)
experiments = [{'config' : r['config'], 'result' : r['result'], 'id' : r['_id']} for r in collection.find() if r['status'] in ('COMPLETED',)]
for ex in experiments:
    with open(ex['result']) as f:
        ex['result'] = json.load(f)

print(f'Number of finished experiments : {len(experiments)}')

Number of finished experiments : 16


In [57]:
def pop_from_metric(metric, d, default=None):
    for k, v in d.items():
        if k in metric:
            return metric.replace(k, ''), v
    else:
        return metric, default
    

In [60]:
SETTING = 'Setting'
OOD_TYPE = 'OOD Experiment'
LOC = 'Leave Out Classes'
PERTURBATIONS = 'Perturbations'
BERNOULLI = 'Bernoulli'
NORMAL = 'Normal'
PROXY = 'Proxy'
RESIDUAL = 'Residual'
SPECTRAL_NORM = 'Spectral Normalization'
NO_EDGES = 'No Edges'
FIT_TO = 'Fit to'
COVARIANCE_TYPE = 'Covariance'

# ---- Settings ----
HYBIRD = 'Hybrid'
TRANSDUCTIVE = 'Transductive'

# ---- Proxies ----
MAX_SCORE = 'Max Score'
PREDICTIVE_ENTROPY = 'Predictive Entropy'
LOGIT_ENERGY = 'Logit Energy'
MOG = 'Gaussian Mixture Model'

# ---- Fitting modes ----
FIT_TRAIN = 'Train'
FIT_ALL = 'All'
FIT_BEST = 'Most confident'
FIT_95_CONF = '95% confident'

# ---- Covariance types ----
DIAGONAL = 'Diagonal'
FULL = 'Full'

# ---- Specifics ----
INITIALIZATION = 'Initialization' # For MoG

In [63]:
data = []
for ex in experiments:
    if ex['config']['data']['setting'] == 'hybrid':
        setting = HYBIRD
    elif ex['config']['data']['setting'] == 'transductive':
        setting = TRANSDUCTIVE
    else:
        raise ValueError(ex['config']['data']['setting'])
    residual = ex['config']['model']['residual']
    spectral_norm = ex['config']['model']['use_spectral_norm']
    
    for idx in range(ex['config']['data']['num_dataset_splits'] * ex['config']['model']['num_initializations']):
        for metric in ex['result'].keys():
            if not 'auroc' in metric:
                continue
            auroc = ex['result'][metric][idx]
            metric = metric.replace('auroc_', '')
            
            
            metric, no_edges = pop_from_metric(metric, {'-no-edges' : True}, default=False)
            metric, ood_type = pop_from_metric(metric, {
                '_loc' : LOC, '_ber' : BERNOULLI, '_normal' : NORMAL
            }, default=None)
            if ood_type is None:
                raise ValueError(f'Unknown ood type {metric}')
                
            metric, fit_to = pop_from_metric(metric, {
                '-fit-all' : FIT_ALL,
                '-fit-best' : FIT_BEST,
                '-fit-95-conf' : FIT_95_CONF,
            }, default=FIT_TRAIN)
            
            # Strip away dim reduction (not used)
            metric, _ = pop_from_metric(metric, {':no' : None})
            metric, covariance_type = pop_from_metric(metric, {'-diag' : DIAGONAL, '-full' : FULL})
            print(metric)
                
            if metric == 'max-score':
                proxy_dict = {
                    PROXY : MAX_SCORE
                }
            elif metric == 'total-predictive-entropy':
                proxy_dict = {
                    PROXY : PREDICTIVE_ENTROPY
                }
            elif metric == 'logit-energy':
                proxy_dict = {
                    PROXY : LOGIT_ENERGY,
                }
            elif metric.startswith('mog'):
                # Mixture of Gaussians
                metric, initialization = pop_from_metric(metric, {'-random' : 'Random', '-predictions' : 'Predicted Scores'}, None)
                assert initialization and covariance_type
                tokens = metric.split('-')
                assert tokens[0] == 'mog'
                proxy_dict = {
                    PROXY : MOG,
                    COVARIANCE_TYPE : covariance_type,
                    INITIALIZATION : initialization,
                }
                
                
            else:
                raise ValueError(metric)
                
            data.append({
                SETTING : setting,
                RESIDUAL : residual,
                SPECTRAL_NORM : spectral_norm,
                NO_EDGES : no_edges,
                OOD_TYPE : ood_type,
            } | proxy_dict)
            
                
            
            
        

total-predictive-entropy
max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
total-predictive-entropy
max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
total-predictive-entropy
max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
total-predictive-entropy
max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mo

mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
total-predictive-entropy
max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
total-predictive-entropy
max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
total-predictive-entropy
max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
m

mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
total-predictive-entropy
max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
total-predictive-entropy
max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
total-predictive-entropy
max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-

mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
total-predictive-entropy
max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
total-predictive-entropy
max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
total-predictive-entropy
max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-

mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
total-predictive-entropy
max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
total-predictive-entropy
max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
total-predictive-entropy
max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
m

...

max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
total-predictive-entropy
max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
total-predictive-entropy
max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
total-predictive-entropy
max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predicti

mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
total-predictive-entropy
max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
total-predictive-entropy
max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
total-predictive-entropy
max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-

mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
total-predictive-entropy
max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
total-predictive-entropy
max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
total-predictive-entropy
max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
m

mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
total-predictive-entropy
max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
total-predictive-entropy
max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
total-predictive-entropy
max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-

total-predictive-entropy
max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
total-predictive-entropy
max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
total-predictive-entropy
max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
total-predictive-entropy
max-score
logit-energy
mog-7-random
mog-7-predictions
mog-7-random
mog-7-predictions
mo

In [67]:
[k for k in experiments[1]['result'].keys()]

['val_loss-val-0',
 'val_accuracy-val-0',
 'accuracy_val',
 'accuracy_id_val',
 'accuracy_ood_val',
 'accuracy_val_no-edges',
 'accuracy_id_val_no-edges',
 'accuracy_ood_val_no-edges',
 'ece_val',
 'ece_val_no-edges',
 'empirical_lipschitz_val_slope_mean_perturbation_noise',
 'empirical_lipschitz_val_slope_median_perturbation_noise',
 'empirical_lipschitz_val_slope_max_perturbation_noise',
 'empirical_lipschitz_val_slope_min_perturbation_noise',
 'empirical_lipschitz_val_slope_mean_perturbation_derangement',
 'empirical_lipschitz_val_slope_median_perturbation_derangement',
 'empirical_lipschitz_val_slope_max_perturbation_derangement',
 'empirical_lipschitz_val_slope_min_perturbation_derangement',
 'accuracy_ood-val_loc-no-edges',
 'accuracy_id_ood-val_loc-no-edges',
 'accuracy_ood_ood-val_loc-no-edges',
 'auroc_total-predictive-entropy_loc-no-edges',
 'aucpr_total-predictive-entropy_loc-no-edges',
 'auroc_max-score_loc-no-edges',
 'aucpr_max-score_loc-no-edges',
 'auroc_logit-energy_lo